# Data Cleaning and Conversion for Call Disposition Model

This notebook cleans the raw Excel data from `data/raw/missing_data.xlsx` and converts it into the structured JSON format required for training.

**Note:** For bulk processing of 100k+ rows, the Python script `preprocess/run_cleaning.py` is faster and recommended.

In [ ]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
import re
import os

# File paths (relative to notebook location)
INPUT_FILE = '../data/raw/missing_data.xlsx'
OUTPUT_FILE = '../data/calls_data.json'

print(f"Loading data from {INPUT_FILE}...")
df = pd.read_excel(INPUT_FILE)
print(f"Total rows loaded: {len(df)}")

## 1. Cleaning Utilities

In [ ]:
def clean_val(val):
    """Deep cleaning of a single value for JSON serializability."""
    if pd.isna(val) or str(val).strip().lower() in ['nat', 'nan', 'null', 'none', '']:
        return None
    if isinstance(val, (pd.Timestamp, np.datetime64)) or 'datetime' in str(type(val)).lower():
        try:
            return str(val).split(' ')[0]
        except:
            return None
    if isinstance(val, (np.integer, int)):
        return int(val)
    if isinstance(val, (np.floating, float)):
        return float(val)
    return val

def format_transcript(text):
    if not isinstance(text, str):
        return ""
    # Standardize Speaker labels
    text = text.replace("Speaker 0:", "Agent:")
    text = text.replace("Speaker 1:", "Borrower:")
    # Remove any other Speaker labels (Speaker 2, Speaker 3, etc.)
    text = re.sub(r"Speaker \d+:", "", text)
    # Clean redundant spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

def map_payment_disposition(val):
    if pd.isna(val): return None
    v = str(val).upper().strip()
    if v in ['PTP', 'WILLING_TO_PAY', 'PROMISE_TO_PAY']: return 'PTP'
    if v in ['PAID', 'ALREADY_PAID_BEFORE_CALL', 'FULL_PAID_ON_CALL', 'PAYMENT_COMPLETED']: return 'PAID'
    if v in ['SETTLEMENT', 'WANT_TO_SETTLE', 'SETTLED_AFTER_CALL', 'ALREADY_SETTLED_BEFORE_CALL', 'SETTLEMENT_NEGOTIATION']: return 'SETTLEMENT'
    if v in ['PARTIAL_PAYMENT', 'PARTIALLY_PAID_AFTER_CALL']: return 'PARTIAL_PAYMENT'
    return val

def clean_remarks(remarks):
    if not remarks or not isinstance(remarks, str) or "Synthetic" in remarks:
        return None
    return remarks

## 2. Process and Filter Data

In [ ]:
INSTRUCTION = """You are an AI assistant that extracts structured call disposition data.
Given a call transcript between an agent and a borrower, extract the following fields. Return ONLY valid JSON. Do not explain.:
disposition, payment_disposition, reason_for_not_paying, ptp_amount, ptp_date, followup_date, remarks.
Current Date: 2026-01-27
If a field is not present, return null."""

json_data = []
dropped_count = 0

print("Processing, cleaning, and filtering rows...")
for _, row in tqdm(df.iterrows(), total=len(df)):
    disposition = clean_val(row['disposition'])
    transcription = format_transcript(row['transcription'])
    
    # Filter: Remove if disposition or transcription is null/empty
    if not disposition or not transcription or transcription.strip() == "":
        dropped_count += 1
        continue

    output_obj = {
        "disposition": disposition,
        "payment_disposition": map_payment_disposition(row['payment_disposition']),
        "reason_for_not_paying": clean_val(row['reason_for_not_paying']),
        "ptp_amount": clean_val(row['ptp_amount']),
        "ptp_date": clean_val(row['ptp_date']),
        "followup_date": clean_val(row['followup_date']),
        "remarks": clean_remarks(str(row['remarks'])) if pd.notna(row['remarks']) else None
    }
    
    entry = {
        "instruction": INSTRUCTION,
        "input": transcription,
        "output": output_obj
    }
    json_data.append(entry)

print(f"Dropped {dropped_count} rows due to null transcripts or dispositions.")
print(f"Total entries preserved: {len(json_data)}")

print(f"Saving to {OUTPUT_FILE}...")
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False, default=lambda x: None if pd.isna(x) else str(x))

print("Done!")